# Ignition Simple network

In [1]:
import maestrobubble

In [2]:
# this is the path to your data files
data_path = '../data/bubble_noneutrino_2spec/'

# These is the input/output prefix of your datafile names.
input_prefix = 'react_inputs_*'
output_prefix = 'react_outputs_*'

# Plotfile prefixes, used for visualization purposes.
plotfile_prefix = 'plt_*'

# By default, this package will save your model, logs of the training and testing data during training,
# and plots to a directory. Here you specify that directory.
output_dir = 'testing123/'

# The log file. Everything that is printed during training also goes into this file in case something
# gets interrupted.
log_file = output_dir + "log.txt"

In [3]:
# We first remove the directory we just generated. If you don't do this you'll get an error. This is to protect
# this package from overwriting your data in case one forgets to change the output_dir when training a new model

!rm -r testing123/

In [4]:
from maestrobubble.train import NuclearReactionML
nrml = NuclearReactionML(data_path, input_prefix, output_prefix, plotfile_prefix,
                output_dir, log_file, DEBUG_MODE=False, DO_PLOTTING=True,
                SAVE_MODEL=True, DO_HYPER_OPTIMIZATION=False, LOG_MODE=False)

Model starting on : 03/07/2022 21:45:16
input_prefix react_inputs_*
output_prefix react_outputs_*
output_dir testing123/
DEBUG_MODE False
DO_PLOTTING True
DO_HYPER_OPTIMIZATION False
Loading Input Files...
Loading Output Files...
Loaded data successfully!


In [5]:
# the package provides preset networks to use
from maestrobubble.networks import *
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_epochs = 500

def selectModel(model_id = 1):
    if model_id == 1:
        model = Net_tanh(4, 32, 32, 32, 3)
    elif model_id == 2:
        model = U_Net(4, 32, 16, 16, 32, 3)
    elif model_id == 3:
        model = ResNet(4, 16, 32, 32, 16, 3)
    elif model_id == 4:
        model = Cross_ResNet(4, 16, 32, 32, 16, 3)
    elif model_id == 5:
        # model = Deep_Net(4, 12, 12, 12, 12, 12, 12, 12, 3)
        # model = Combine_Net3(4, 16, 8, 8, 8, 8, 8, 8, 8, 8, 16, 3)
        model = Combine_Net3(4, 32, 16, 8, 8, 8, 8, 8, 8, 16, 32, 3)
    else:
        model = Net(4, 16, 16, 16, 3)
    # get model to cuda if possible
    model.to(device=device)
    return model

cuda


In [6]:
def loadModel(model_id, model_path):
    if model_id == 1:
        model = Net_tanh(4, 32, 32, 32, 3)
    elif model_id == 2:
        model = U_Net(4, 32, 16, 16, 32, 3)
    elif model_id == 3:
        model = ResNet(4, 16, 32, 32, 16, 3)
    elif model_id == 4:
        model = Cross_ResNet(4, 16, 32, 32, 16, 3)
    elif model_id == 5:
        # model = Deep_Net(4, 12, 12, 12, 12, 12, 12, 12, 3)
        model = Combine_Net3(4, 32, 16, 8, 8, 8, 8, 8, 8, 16, 32, 3)
    else:
        model = Net(4, 16, 16, 16, 3)

    try:
        model.load_state_dict(torch.load(model_path))
    except RuntimeError:
        model.module.load_state_dict(torch.load(model_path))
    print(model)

    # get model to cuda if possible
    model.to(device)
    return model

In [7]:
# a more complicated loss function with physics constraints
from maestrobubble.losses import log_loss, logX_loss, loss_mass_fraction_sum_L
import torch.nn as nn

def criterion(pred, target): 
    #loss1 = logX_loss(pred, target, nnuc=2)
    #loss2 = 10*loss_mass_fraction_sum_L(pred, totsum=0.3, nnuc=2)
    
    L = nn.MSELoss()
    F = nn.L1Loss()
    return L(pred[:, :3], target[:, :3]) + F(torch.sign(pred[:,2]), torch.sign(target[:,2]))
    #return loss1 #+ loss2

In [8]:
import shutil
import os

def moveData(model_id):
    subdir = output_dir + "model" + str(model_id) + "/"
    os.mkdir(subdir)
    shutil.move(output_dir + "component_losses_test.txt", subdir + "component_losses_test.txt")
    shutil.move(output_dir + "component_testing_loss.png", subdir + "component_testing_loss.png")
    shutil.move(output_dir + "component_losses_train.txt", subdir + "component_losses_train.txt")
    shutil.move(output_dir + "component_training_loss.png", subdir + "component_training_loss.png")
    shutil.move(output_dir + "cost_per_epoch.txt", subdir + "cost_per_epoch.txt")
    shutil.move(output_dir + "cost_vs_epoch.png", subdir + "cost_vs_epoch.png")
    shutil.move(output_dir + "my_model.pt", subdir + "my_model.pt")
    shutil.move(output_dir + "prediction_vs_solution_log.png", subdir + "prediction_vs_solution_log.png")
    shutil.move(output_dir + "prediction_vs_solution.png", subdir + "prediction_vs_solution.png")
    

In [ ]:
import torch.optim as optim
from IPython.display import clear_output

for i in range(0,6):
    clear_output()
    print(f"Model {i} \n")
    
    model = selectModel(i)
#     model_path = "testing123/model" + str(i) + "/my_model.pt"
#     model = loadModel(i, model_path)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    nrml.train(model, optimizer, num_epochs, criterion)
    
    # need to put model on cpu for plotting
    #model.to(device=torch.device("cpu"))

    nrml.plot()
    
    moveData(i)
    !rm -r testing123/intermediate_output/
    
# !mv testing123 testing123_done

Model 0 

Cost at epoch 0 is 0.6699403848409934


In [9]:
#load model (option)
from maestrobubble.networks import Net_tanh, Cross_ResNet, Deep_ResNet, Combine_Net3, U_Net
import torch

model_path = "testing123/" + "my_model.pt"
#model = Net_tanh(5, 16, 16, 16, 16, 4)
#model = Cross_ResNet(5, 16, 16, 16, 16, 4)
#model = Deep_ResNet(5, 8, 8, 8, 8, 8, 8, 8, 8, 4)
#model = Combine_Net3(4, 16, 8, 8, 8, 8, 8, 8, 8, 8, 16, 3)
model = Combine_Net3(4, 32, 16, 8, 8, 8, 8, 8, 8, 16, 32, 3)
#model = Net_tanh(5, 32, 64, 32, 4)
#model = U_Net(4, 32, 8, 8, 16, 3)
model.load_state_dict(torch.load(model_path))
print(model)

Combine_Net3(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (ac1): Tanh()
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (ac2): Tanh()
  (fc3): Linear(in_features=8, out_features=8, bias=True)
  (ac3): Tanh()
  (fc4): Linear(in_features=8, out_features=8, bias=True)
  (ac4): Tanh()
  (fc5): Linear(in_features=8, out_features=8, bias=True)
  (ac5): Tanh()
  (fc6): Linear(in_features=8, out_features=8, bias=True)
  (ac6): Tanh()
  (fc7): Linear(in_features=8, out_features=8, bias=True)
  (ac7): Tanh()
  (fc8): Linear(in_features=8, out_features=8, bias=True)
  (ac8): Tanh()
  (fc9): Linear(in_features=8, out_features=8, bias=True)
  (ac9): Tanh()
  (fc10): Linear(in_features=8, out_features=16, bias=True)
  (ac10): Tanh()
  (fc11): Linear(in_features=16, out_features=3, bias=True)
  (fc1to4): Linear(in_features=16, out_features=8, bias=True)
  (fc3to6): Linear(in_features=8, out_features=8, bias=True)
  (fc5to8): Linear(in_features=8, out_features=8, bias=Tru

In [9]:
# a more complicated loss function with physics constraints
from maestrobubble.losses import logX_loss, loss_mass_fraction_sum_L
import torch.nn as nn

def criterion(pred, target): 
    #loss1 = logX_loss(pred, target)
    loss2 = 10*loss_mass_fraction_sum_L(pred, totsum=0.3, nnuc=2)
    
    L = nn.MSELoss()
    F = nn.L1Loss()
    return loss2 + L(pred[:, :3], target[:, :3]) + F(torch.sign(pred[:,2]), torch.sign(target[:,2]))
    #return loss1 #+ loss2

In [ ]:
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_epochs = 1000

# get model to cuda if possible
model.to(device=device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

nrml.train(model, optimizer, num_epochs, criterion)

In [11]:
# need to put model on cpu for plotting
#model.to(device=torch.device("cpu"))

nrml.plot()

Plotting...


In [ ]:
import glob
plots = glob.glob('testing123/*.png')
from IPython.display import Image, display

for plot in plots:
    fig = Image(filename=(plot))
    display(fig)

In [10]:
# convert to torch script
net_module = torch.jit.script(model)
net_module.save("ts_model.pt")
print(net_module.code)

def forward(self,
    x: Tensor) -> Tensor:
  x1 = (self.ac1).forward((self.fc1).forward(x, ), )
  x2 = (self.ac2).forward((self.fc2).forward(x1, ), )
  x3 = (self.ac3).forward((self.fc3).forward(x2, ), )
  _0 = self.ac4
  _1 = torch.add((self.fc4).forward(x3, ), (self.fc1to4).forward(x1, ))
  x4 = (_0).forward(_1, )
  x5 = (self.ac5).forward((self.fc5).forward(x4, ), )
  _2 = self.ac6
  _3 = torch.add((self.fc6).forward(x5, ), (self.fc3to6).forward(x3, ))
  x6 = (_2).forward(_3, )
  x7 = (self.ac7).forward((self.fc7).forward(x6, ), )
  _4 = self.ac8
  _5 = torch.add((self.fc8).forward(x7, ), (self.fc5to8).forward(x5, ))
  x8 = (_4).forward(_5, )
  x9 = (self.ac9).forward((self.fc9).forward(x8, ), )
  _6 = self.ac10
  _7 = torch.add((self.fc10).forward(x9, ), (self.fc7to10).forward(x7, ))
  _8 = torch.add(_7, (self.fc1to10).forward(x1, ))
  x10 = (_6).forward(_8, )
  x11 = torch.add((self.fc11).forward(x10, ), (self.fcio).forward(x, ))
  return x11



In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.cla()
colors = matplotlib.cm.rainbow(np.linspace(0, 1, 3))
print(colors.shape)
colors = np.tile(colors, (2,1))
print(colors)
x = [[2, 3, 6], [4, 4, 5]]
y = [[1, 2, 5], [3, 3, 3]]
plt.scatter(x, y, c=colors)
plt.show()

In [ ]:
data, targets = next(iter(nrml.train_loader))
print(f"Data batch shape: {data.size()}")
print(f"Targets batch shape: {targets.size()}")

In [ ]:
model.to(device=torch.device("cpu"))
pred = model(data)
print(f"Prediction batch shape: {pred.size()}")
print(f"Targets batch shape: {targets.size()}")

In [ ]:
-2.0/torch.log(torch.tensor([0.8, 10**-12]))

In [ ]:
-2.0/torch.log(torch.tensor([-0.0]))

In [14]:
import numpy as np
import torch
iout = [i for i in range(4)]
iout = [0, 2, 3]
x = np.array([0, 1, 2, 3, 4])
xt = torch.from_numpy(x)
xt[iout]
xt.to(device=torch.device("cuda:0"))
print(f"GPU {torch.cuda.current_device()} of {torch.cuda.device_count()} available devices")
print(device)

GPU 0 of 1 available devices
cuda


In [16]:
print(xt)
xt = xt.to(dtype=torch.float)
xt.reshape((1,xt.shape[0]))

tensor([0., 1., 2., 3., 4.])


tensor([[0., 1., 2., 3., 4.]])